**Set environment**

In [ ]:
source ../config/config_duke.sh
source ../config/config_load_module_bedtools.sh
show_env

**Check data**

In [ ]:
FD_REG=${FD_RES}/results/region/annotation_ccres
FN_REG=ccres_v4.bed.gz

echo +++++++++++++++++++++++++++++++++++
ls -lh ${FD_REG}/${FN_REG}
fun_head ${FD_REG}/${FN_REG} | wc -l
echo

echo +++++++++++++++++++++++++++++++++++
fun_head ${FD_REG}/${FN_REG}
echo

#echo +++++++++++++++++++++++++++++++++++
#tail ${FD_REG}/${FN_REG}
#echo

In [ ]:
ASSAY=A001_K562_WSTARRseq
FD_INP=${FD_RES}/results/${ASSAY}/fragment_count

echo ${FD_INP}
ls ${FD_INP}/*WGS*.bed.gz | xargs -n 1 basename

In [ ]:
FP_INPS=($(ls ${FD_INP}/*WGS*.bed.gz))
echo Total: ${#FP_INPS[@]}

**RUN**

In [5]:
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 8 \
    --mem 20G \
    --output ${FD_LOG}/coverage_ccres_wstarr.%a.txt \
    --array 0-6 \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh
ASSAY=A001_K562_WSTARRseq

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### set input and output directory
FD_INP=${FD_RES}/results/${ASSAY}/fragment_count
FD_OUT=${FD_RES}/results/${ASSAY}/coverage_ccres_v4

FP_INPS=($(ls ${FD_INP}/*WGS*.bed.gz))
FP_INP=${FP_INPS[${SLURM_ARRAY_TASK_ID}]}
FN_INP=$(basename ${FP_INP})

FN_OUT=${FN_INP}
FP_OUT=${FD_OUT}/${FN_OUT}

FD_REG=${FD_RES}/results/region/annotation_ccres
FN_REG=ccres_v4.bed.gz
FP_REG=${FD_REG}/${FN_REG}

### Show I/O file
echo "Input  file: " ${FP_INP}
echo
echo "Show the first few lines of the input file"
fun_head ${FP_INP}
echo

echo "Input  file: " ${FP_REG}
echo
echo "Show the first few lines of the input file"
fun_head ${FP_REG}
echo

### execute
bedtools map \
    -a ${FP_REG} \
    -b ${FP_INP} \
    -o sum \
    -f 0.5 \
    -F 0.5 \
    -e \
| gzip -c \
> ${FP_OUT}

### Show I/O file
echo
echo "Output file: " ${FP_OUT}
echo
echo "Show the first few lines of the output file"
fun_head ${FP_OUT}
echo

### print end message
timer_stop=`date +%s`
runtime=$(echo "${timer_stop} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 30632739


**Check results**

In [7]:
cat ${FD_LOG}/coverage_ccres_wstarr.0.txt

Hostname:           x2-04-2.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         01-18-24+16:23:10

Input  file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/A001_K562_WSTARRseq/fragment_count/A001_K562_WSTARRseq.Input.rep1.WGS.unstranded.bed.gz

Show the first few lines of the input file
chr1	10335	10549	chr1_10335_10549	1	.
chr1	16983	17500	chr1_16983_17500	1	.
chr1	16985	17499	chr1_16985_17499	1	.
chr1	17004	17500	chr1_17004_17500	1	.
chr1	17483	17898	chr1_17483_17898	1	.
chr1	17487	17941	chr1_17487_17941	1	.
chr1	33606	33997	chr1_33606_33997	1	.
chr1	33617	33997	chr1_33617_33997	1	.
chr1	38785	39226	chr1_38785_39226	1	.
chr1	46328	46678	chr1_46328_46678	1	.

Input  file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_ccres/ccres_v4.bed.gz

Show the first few lines of the input file
chr1	10033	10250	EH38E2776516	0	.	10033	10250	225,225,225	Low-DNase	All-data/Full-classification
chr1	10385	10713	EH38E2776517	0	.	10385	10713	225,225,2